## Carga informacion

In [1]:
import glob
import os
import pandas as pd
import warnings
from pathlib import Path

# Ocultar warnings de openpyxl sobre headers/footers
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# ==========
# 1) Unir los 8 Excels de septiembre
# ==========
ruta_ns = r"/home/donsson/proyectos/INDICADOR NS/dataset/ns_septiembre"
archivos_ns = glob.glob(os.path.join(ruta_ns, "*.xlsx"))

if not archivos_ns:
    raise FileNotFoundError(f"No se encontró ningún archivo en {ruta_ns}")

# Unir todos los Excels en un solo DataFrame
df_unido = pd.concat([pd.read_excel(f) for f in archivos_ns], ignore_index=True)

print(f"✅ Se unieron {len(archivos_ns)} archivos de septiembre con {df_unido.shape[0]} filas totales")

# ==========
# 2) Cargar inventario limpio
# ==========
ruta_inventario = "/home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx"

if not os.path.exists(ruta_inventario):
    raise FileNotFoundError(f"No existe el archivo {ruta_inventario}")

df_inventario = pd.read_excel(ruta_inventario)

print(f"📦 Inventario cargado con {df_inventario.shape[0]} filas desde: {ruta_inventario}")



✅ Se unieron 8 archivos de septiembre con 38922 filas totales
📦 Inventario cargado con 14841 filas desde: /home/donsson/proyectos/INVENTARIO/inventario_limpio.xlsx


In [2]:
df_unido.sample(5)

,Sucursal,Marca,Clase,Producto,Descripcion,Enero nv,Febrero nv,Marzo nv,Abril nv,Mayo nv,...,Marzo vp,Abril vp,Mayo vp,Junio vp,Julio vp,Agosto vp,Septiembre vp,Octubre vp,Noviembre vp,Diciembre vp
11747,SUCURSAL BARRANQUILLA,OTROS,C,DAE61013139,[DAE61013139] GA1013 FILTRO AIRE HYUNDAI VISIO...,1.0,1.0,1.0,1.000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11425,SUCURSAL BARRANQUILLA,DONSSON,C,DAX92021222,[DAX92021222] GXKL21 CARCASA 735CFM. Lleva DA2...,1.0,1.0,1.0,1.000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34972,SUCURSAL BUCARAMANGA,DONSSON,A,DAE09142025,[DAE09142025] DA9142 FILTRO AIRE DE SEGURIDAD...,NaN,NaN,NaN,0.936,1.0,...,10.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11644,SUCURSAL BARRANQUILLA,AUT*PARTS,C,DAE05776189,"[DAE05776189] GA776 FILTRO AIRE TOYOTA TERCEL,...",1.0,1.0,1.0,1.000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22539,SUCURSAL NORTE,BALDWIN,C,BHS00452125,[BHS00452125] GS452 FILTRO HIDRAULICO JOHN DEE...,1.0,1.0,1.0,1.000,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_sin_clase = df_unido[df_unido["Clase"]=="SIN CLASE X SUCURSAL"]  # identificar productos sin categoria y revisar si son nuevos

df_sin_clase = df_sin_clase.groupby("Descripcion")["Producto"].count()
df_sin_clase.sample(5)

Descripcion
[DAB08279025] DA8279 FILTRO AIRE - JOHN DEERE - HYUNDAI (025 DA8279)              8
[DAB09279025] DA9279 FILTRO AIRE SEGURIDAD - JOHN DEERE - HYUNDAI (025 DA9279)    8
[DAC00243020] DAC243 FILTRO AIRE CABINA - JOHN DEERE (020 DAC243)                 2
[OTROS] OTROS COBROS EN SUMINISTRO DE FILTROS                                     8
[ACUEDUCTO Y ALCANTARILLADO] ACUEDUCTO Y ALCANTARILLADO                           8
Name: Producto, dtype: int64

## Limpieza

In [4]:
from datetime import datetime
import re

def limpiar_y_transformar(df_unido):
    # 1. Obtener el mes actual en español manualmente
    meses_es = [
        "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
        "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
    ]
    mes_actual = meses_es[datetime.now().month - 1]

    # 2. Quitar columnas desde el mes siguiente al actual
    meses_nv = [f"{mes.lower()} nv" for mes in meses_es]
    meses_vp = [f"{mes.lower()} vp" for mes in meses_es]

    idx_mes_actual = meses_es.index(mes_actual)

    # Columnas a eliminar: desde el mes siguiente hasta diciembre
    meses_a_borrar = meses_nv[idx_mes_actual+1:] + meses_vp[idx_mes_actual+1:]
    columnas_a_eliminar = [col for col in df_unido.columns if col.lower() in meses_a_borrar]
    df_unido = df_unido.drop(columns=columnas_a_eliminar, errors='ignore')

    # 3. Clasificación categoría
    marcas_filtros = [ "BALDWIN", "DONSSON", "AUT*PARTS", "RAMA", "RACOR USA", "RACOR BRASIL"
    ]

    #marcas_aceite_lubricantes = []

    def clasificar_categoria(marca):
        marca = str(marca).upper().strip()
        if marca == "OTROS":
            return "Otros"
        elif marca in marcas_filtros:
            return "Filtro"
        else:
            return "Aceite/Lubricante"

    df_unido["categoria_producto"] = df_unido["Marca"].apply(clasificar_categoria)



    # 4. Extraer código interno        
    def extraer_codigo(descripcion, categoria):
        if not isinstance(descripcion, str):
            return None

    # Para Aceite/Lubricante, tomar directamente lo que está dentro de los corchetes
        if categoria == "Aceite/Lubricante":
            match = re.search(r"\[([A-Z0-9\-]+)\]", descripcion)
            if match:
                return match.group(1)

    # Para Filtros, aplicar la lógica anterior

    
        elif categoria == "Filtro" or categoria == "Otros":
            match = re.search(r"\[([A-Z0-9\-]+)\]\s+(\S+)", descripcion)
            if match:
                codigo_base = match.group(1)
                palabra_siguiente = match.group(2)
            
            # Tomar los últimos 3 dígitos (no caracteres): buscar solo dígitos desde el final
            ultimos_digitos = ''.join(re.findall(r'\d', codigo_base))[-3:]
            return f"{ultimos_digitos} {palabra_siguiente}"

        

    df_unido["Codigo_interno"] = df_unido.apply(
    lambda row: extraer_codigo(row["Descripcion"], row["categoria_producto"]),
    axis=1)
    df_unido['Codigo_Numerico'] = df_unido['Codigo_interno'].str.split(' ').str[0]


    # 5. Definir las listas de códigos numéricos para filtros
    importacion = ['189', '125', '138', '137', '222', '225']
    nacional = ['004', '190', '005']
    fabricado = ['025']

    # 5. Crear una función de clasificación que considera la categoría y el código numérico
    def clasificar_producto(row):
        # Si la categoría es "Filtro", usamos las listas para clasificar
        if row['categoria_producto'] == 'Filtro':
            if row['Codigo_Numerico'] in importacion:
                return 'Importacion'
            elif row['Codigo_Numerico'] in nacional:
                return 'Nacional'
            elif row['Codigo_Numerico'] in fabricado:
                return 'Fabricado'
            else:
                return 'Desconocido' # Si es un filtro con un código no listado
            # Para cualquier otra categoría (Aceite/Lubricante, etc.), clasificar como "Nacional"
        else:
            return 'Nacional'
        
    df_unido['Tipo_origen'] = df_unido.apply(clasificar_producto, axis=1)

                                

    # 6. Eliminar productos con clase "SIN CLASE X SUCURSAL" y aquellos que no tienen marca
     
    df_unido = df_unido[~(df_unido['Marca'].isna() | (df_unido['Marca'].str.strip() == ''))]

    df_unido = df_unido[df_unido["Clase"] != "SIN CLASE X SUCURSAL"]

    df_unido["Sucursal"] = df_unido["Sucursal"].replace("PRINCIPAL COTA", "MOSTRADOR COTA") #Correccion bodega 8 esatba como principal cota
    df_unido["Sucursal"] = df_unido["Sucursal"].str.replace("Sucursal ", "", regex=False) #Dejar sin la palabra sucursal




    # 7. Agregar columna con la meta de nivel de servicio según tipo/clase
    metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.95,
        'C': 0.50
    }

    # Normalizar la columna de clase por si hay espacios o minúsculas
    df_unido["Clase"] = df_unido["Clase"].str.strip().str.upper()

    # Crear columna con la meta
    df_unido["meta_ns"] = df_unido["Clase"].map(metas_nv)

    # 8. Comparar si se cumplió la meta
    # Asegúrate de tener una columna real con el nivel de servicio (ajusta el nombre si es otro)
    if "nivel_servicio" in df_unido.columns:
        df_unido["cumple_meta"] = df_unido["nivel_servicio"] >= df_unido["meta_ns"]




    return df_unido




In [5]:
df_limpio = limpiar_y_transformar(df_unido)
df_limpio = df_limpio[['Sucursal' ,'categoria_producto','Producto', 'Marca' , 'Clase', 'Codigo_interno' , 'Septiembre nv', 'Septiembre vp','meta_ns','Tipo_origen','Descripcion']]
df_limpio.head()

,Sucursal,categoria_producto,Producto,Marca,Clase,Codigo_interno,Septiembre nv,Septiembre vp,meta_ns,Tipo_origen,Descripcion
2,SUCURSAL CALLE 6,Filtro,BLS00781125,BALDWIN,C,125 GS781,NaN,4.0,0.5,Importacion,[BLS00781125] GS781 FILTRO ACEITE - CUMMINS (1...
3,SUCURSAL CALLE 6,Filtro,BAR02273125,BALDWIN,C,125 DA2273,1.0,NaN,0.5,Importacion,[BAR02273125] DA2273 FILTRO AIRE - INGERSOLL R...
4,SUCURSAL CALLE 6,Filtro,BAB12872125,BALDWIN,C,125 DA2872A,1.0,NaN,0.5,Importacion,[BAB12872125] DA2872A FILTRO AIRE - CATERPILLA...
5,SUCURSAL CALLE 6,Filtro,BAB02612125,BALDWIN,C,125 DA2612,1.0,NaN,0.5,Importacion,[BAB02612125] DA2612 FILTRO AIRE - TOYOTA (125...
6,SUCURSAL CALLE 6,Filtro,BAR09273125,BALDWIN,C,125 DA9273,1.0,NaN,0.5,Importacion,[BAR09273125] DA9273 FILTRO AIRE SEGURIDAD - C...


In [6]:
df_limpio.groupby(['Sucursal','Clase'])['Septiembre vp'].sum()

Sucursal               Clase
MOSTRADOR COTA         A         440.0
                       AAA       350.0
                       B         443.0
                       C        3063.0
SUCURSAL BARRANQUILLA  A         215.0
                       AAA         0.0
                       B         359.0
                       C        1121.0
SUCURSAL BUCARAMANGA   A         157.0
                       AAA        45.0
                       B         151.0
                       C         734.0
SUCURSAL CALI          A         323.0
                       AAA        77.0
                       B         188.0
                       C        1301.0
SUCURSAL CALLE 6       A         481.0
                       AAA        77.0
                       B         430.0
                       C         806.0
SUCURSAL MEDELLIN      A         126.0
                       AAA        39.0
                       B          79.0
                       C         995.0
SUCURSAL NORTE         A          8

### Puedo ver las ventas perdidas de todos los meses que han trasncurrido

In [7]:
df_completo = limpiar_y_transformar(df_unido)
df_completo_vp = df_completo[['Sucursal' ,'categoria_producto', 'Marca' , 'Clase', 'Codigo_interno' ,'Enero vp','Febrero vp','Marzo vp','Mayo vp','Abril vp','Junio vp','Julio vp','Agosto vp','Septiembre vp','meta_ns','Tipo_origen','Descripcion']]
df_largo = df_completo_vp.melt(
    id_vars=['Sucursal', 'categoria_producto', 'Marca', 'Clase', 'Codigo_interno', 'meta_ns', 'Tipo_origen', 'Descripcion'],
    value_vars=['Enero vp', 'Febrero vp', 'Marzo vp', 'Abril vp', 'Mayo vp', 'Junio vp', 'Julio vp', 'Agosto vp', 'Septiembre vp'],
    var_name='Mes',
    value_name='Ventas Perdidas'
)


# Quitar el " vp" del final
df_largo['Mes'] = df_largo['Mes'].str.replace(' vp', '', regex=False)

# Diccionario para ordenar meses
orden_meses = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4,
    'Mayo': 5, 'Junio': 6, 'Julio': 7, 'Agosto': 8,
    'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}

# Columna auxiliar de orden
df_largo['Mes_num'] = df_largo['Mes'].map(orden_meses)


# Primero eliminar filas que no aportan nada
# Agrupamos por producto y verificamos si TODOS los valores de "Ventas Perdidas" son NaN o 0
mask = df_largo.groupby("Codigo_interno")["Ventas Perdidas"].transform(
    lambda x: x.notna().any() and x.fillna(0).sum() > 0
)

# Filtrar solo los productos que sí tienen al menos una venta perdida
df_filtrado = df_largo[mask]

df_agosto = df_filtrado[df_filtrado["Mes"]=="Septiembre"]

df_agosto.to_excel("vp_septiembre.xlsx")

# Guardar para BI
df_filtrado.to_excel("v_perdidas/ventas_perdidas_general.xlsx", index=False)




## Unir DF costo

In [8]:
df_inv_costos = df_inventario[['Codigo_interno', 'Sucursal', 'Costo unitario']].drop_duplicates()

df_merged = df_limpio.merge( 
    df_inv_costos,
    on = ['Codigo_interno','Sucursal'], 
    how='left'
)

df_merged['costo_vp'] = df_merged['Septiembre vp'] * df_merged['Costo unitario']

df_limpio= df_merged

In [9]:
df_limpio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38568 entries, 0 to 38567
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Sucursal            38568 non-null  object 
 1   categoria_producto  38568 non-null  object 
 2   Producto            38568 non-null  object 
 3   Marca               38568 non-null  object 
 4   Clase               38568 non-null  object 
 5   Codigo_interno      38568 non-null  object 
 6   Septiembre nv       36370 non-null  float64
 7   Septiembre vp       3282 non-null   float64
 8   meta_ns             38568 non-null  float64
 9   Tipo_origen         38568 non-null  object 
 10  Descripcion         38568 non-null  object 
 11  Costo unitario      8848 non-null   float64
 12  costo_vp            923 non-null    float64
dtypes: float64(5), object(8)
memory usage: 3.8+ MB


In [10]:


# Suponiendo que ya tienes tu df_limpio
# df_limpio = cargar_datos(str(ruta_carpeta))

# 1) Pedir el mes al usuario
mes = input("\nIngrese el mes (ejemplo: julio): ").strip().lower()

# 2) Construir la ruta completa del archivo
nombre_archivo = Path("informes") / f"datos_{mes}.xlsx"

# 3) Guardar el DataFrame en la carpeta 'informes'
df_limpio.to_excel(nombre_archivo, index=False)

print(f"\n✓ Archivo guardado como {nombre_archivo}")



✓ Archivo guardado como informes/datos_septiembre.xlsx


In [11]:
vp_general = df_limpio[df_limpio['Septiembre vp']>0]
vp_general.to_excel(f"v_perdidas/ventas_perdidas_{mes}.xlsx",index=False)

# Indicador NS

In [12]:
metas_nv = {
        'AAA': 0.999,
        'A': 0.98,
        'B': 0.95,
        'C': 0.50
    }

def calcular_nivel_servicio(df, mes=None):
    # Paso 1: Identificar columnas de nivel de servicio
    columnas_nv = [col for col in df.columns if col.endswith('nv')]
    
    # Paso 2: Ordenar las columnas por orden real de meses
    orden_meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                   'Julio','Agosto','Septiembre']
    
    columnas_nv_ordenadas = sorted(
        columnas_nv,
        key=lambda x: orden_meses.index(x.split()[0])
    )

    # Paso 3: Si no se especifica el mes, detectar el último con al menos 80% de datos
    if mes is None:
        mes_final = None
        for col in reversed(columnas_nv_ordenadas):
            if df[col].notna().mean() > 0.8:
                mes_final = col
                break
        if mes_final is None:
            raise ValueError("No hay columnas de nivel de servicio con suficientes datos.")
    else:
        mes_final = next((col for col in columnas_nv if col.startswith(mes)), None)
        if mes_final is None:
            raise ValueError(f"No se encontró una columna de nivel de servicio para el mes '{mes}'.")

    # Paso 4: Calcular promedio por sucursal y clase
    resultado = (
        df.groupby(['Sucursal', 'Clase'])[mes_final]
        .mean()
        .reset_index()
        .rename(columns={mes_final: 'Nivel_Servicio'})
    )

    # Paso 5: Evaluar contra la meta según clase
    resultado['Meta'] = resultado['Clase'].map(metas_nv)
    resultado['Cumple'] = resultado['Nivel_Servicio'] >= resultado['Meta']
    resultado['Mes'] = mes_final.split()[0]

    return resultado


In [13]:
# Si quieres que el sistema lo escoja automáticamente:
nivel_servicio = calcular_nivel_servicio(df_limpio)

# O si tú decides el mes (por ejemplo, "Junio"):
nivel_servicio = calcular_nivel_servicio(df_limpio, mes="Septiembre")

nivel_servicio

,Sucursal,Clase,Nivel_Servicio,Meta,Cumple,Mes
0,MOSTRADOR COTA,A,0.958189,0.980,False,Septiembre
1,MOSTRADOR COTA,AAA,0.966450,0.999,False,Septiembre
2,MOSTRADOR COTA,B,0.962986,0.950,True,Septiembre
3,MOSTRADOR COTA,C,0.990400,0.500,True,Septiembre
4,SUCURSAL BARRANQUILLA,A,0.982723,0.980,True,Septiembre
5,SUCURSAL BARRANQUILLA,AAA,1.000000,0.999,True,Septiembre
6,SUCURSAL BARRANQUILLA,B,0.957471,0.950,True,Septiembre
7,SUCURSAL BARRANQUILLA,C,0.994678,0.500,True,Septiembre
8,SUCURSAL BUCARAMANGA,A,0.976010,0.980,False,Septiembre
9,SUCURSAL BUCARAMANGA,AAA,0.987146,0.999,False,Septiembre


In [14]:
df_largo.head()

,Sucursal,categoria_producto,Marca,Clase,Codigo_interno,meta_ns,Tipo_origen,Descripcion,Mes,Ventas Perdidas,Mes_num
0,SUCURSAL CALLE 6,Filtro,BALDWIN,C,125 GS781,0.5,Importacion,[BLS00781125] GS781 FILTRO ACEITE - CUMMINS (1...,Enero,NaN,1
1,SUCURSAL CALLE 6,Filtro,BALDWIN,C,125 DA2273,0.5,Importacion,[BAR02273125] DA2273 FILTRO AIRE - INGERSOLL R...,Enero,NaN,1
2,SUCURSAL CALLE 6,Filtro,BALDWIN,C,125 DA2872A,0.5,Importacion,[BAB12872125] DA2872A FILTRO AIRE - CATERPILLA...,Enero,NaN,1
3,SUCURSAL CALLE 6,Filtro,BALDWIN,C,125 DA2612,0.5,Importacion,[BAB02612125] DA2612 FILTRO AIRE - TOYOTA (125...,Enero,NaN,1
4,SUCURSAL CALLE 6,Filtro,BALDWIN,C,125 DA9273,0.5,Importacion,[BAR09273125] DA9273 FILTRO AIRE SEGURIDAD - C...,Enero,NaN,1
